In [24]:
import os
import sys
import glob
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
from random import shuffle


In [20]:
def create_dir(directory):
    """Creates a directory.
    Args:
        directory(str): Path of the directory to create.
    """
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

'Creates a directory.\nArgs:\n    directory(str): Path of the directory to create.\n'

In [21]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively

    Args:
        directory(str): Path of the directory.
    """
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [22]:
# Create directories to separate pictures of cats and dogs based on their labels.
DIRALLDATA = "../input/train"
DIRTRAINCAT = "train/cat"
DIRTRAINDOG = "train/dog"
DIRVALCAT = "val/cat"
DIRVALDOG = "val/dog"

create_dir(DIRTRAINCAT)
create_dir(DIRTRAINDOG)
create_dir(DIRVALCAT)
create_dir(DIRVALDOG)

for img in tqdm(os.listdir(DIRALLDATA)):
    label = img.split('.')[0]
    if label == 'cat':
        if len([name for name in os.listdir(DIRTRAINCAT) if os.path.isfile(os.path.join(DIRTRAINCAT, name))]) < 10000:
            shutil.copy(os.path.join(DIRALLDATA, img), DIRTRAINCAT)
        else:
            shutil.copy(os.path.join(DIRALLDATA, img), DIRVALCAT)
    elif label == 'dog':
        if len([name for name in os.listdir(DIRTRAINDOG) if os.path.isfile(os.path.join(DIRTRAINDOG, name))]) < 10000:
            shutil.copy(os.path.join(DIRALLDATA, img), DIRTRAINDOG)
        else:
            shutil.copy(os.path.join(DIRALLDATA, img), DIRVALDOG)

100%|██████████| 25000/25000 [11:58<00:00, 34.77it/s]


In [30]:
# Data preprocessing and augmentation

train_img = 'train' 
validation_img = 'val'
nb_train_samples = get_nb_files(train_img)
nb_classes = len(glob.glob(train_img + "/*"))

# Parameters for the image augmentation for the train data.
train_datagen = ImageDataGenerator(
    rescale=1./255)
"""    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')"""

validation_datagen = ImageDataGenerator(
    rescale=1./255)

# Image generator.
# Will pick batches of images directly from the directory,
# apply data augmenation on them and label them according 
# to the directory they were picked from.
train_generator = train_datagen.flow_from_directory(
    train_img,
    batch_size=32,
    class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(
    validation_img,
    batch_size=32,
    class_mode='categorical')    

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# CNN classifier with transfer learning from InceptionV3.

input = Input((None, None, 3))
net = InceptionV3(include_top=False, weights='imagenet')(input)
#net = Conv2D(32, 3, padding='same', activation='relu')(net)
net = Conv2D(2, 1)(net)
net = GlobalAveragePooling2D()(net)
output = Activation('softmax')(net)

model = Model(inputs = input, outputs = output)
#Freezing InceptionV3.
model.layers[1].trainable=False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [32]:
# Training and evaluation.
model.fit_generator(train_generator,
                    steps_per_epoch= 20000 // 32,
                    epochs=3,
                    validation_data=validation_generator,
                    validation_steps=100) 

Epoch 1/3
625/625 [==============================] - 177s 283ms/step - loss: 0.1625 - acc: 0.9371 - val_loss: 0.0683 - val_acc: 0.9800
Epoch 2/3
625/625 [==============================] - 165s 264ms/step - loss: 0.1184 - acc: 0.9545 - val_loss: 0.0414 - val_acc: 0.9899
Epoch 3/3
625/625 [==============================] - 165s 264ms/step - loss: 0.1091 - acc: 0.9567 - val_loss: 0.0638 - val_acc: 0.9819
